## 1) กำหนดค่า pathและ threshold

# การตั้งค่า path

*   MODEL_PATH ระบุ path ของโมเดล .onnx ที่ใช้สำหรับ inference
*   IMAGE_PATH ระบุ path ของภาพ input ที่ต้องการตรวจจับวัตถุ
*   SAVE_PATH ระบุ path สำหรับบันทึกภาพผลลัพธ์หลังการตรวจจับ

# การตั้งค่า Input และ Threshold

*   INPUT_SIZE → ขนาด input ของโมเดล (ต้องตรงกับตอน train/export)
*   CONF_THRES → ค่าความเชื่อมั่นขั้นต่ำสำหรับคัดกรองกล่อง
*   IOU_THRES → เกณฑ์ซ้อนทับที่ใช้ใน NMS เพื่อเลือกกล่องที่เหมาะสมที่สุด



In [ ]:
# =====================================================
# 1) Configure Paths and Thresholds
# =====================================================
import onnxruntime as ort
import numpy as np
import cv2

MODEL_PATH = "/content/best.onnx"
IMAGE_PATH = "/content/2024_11_25_15_03_33_exp_5_000000_NAVSTAR38USA126_flipH_blur.png"
SAVE_PATH = "/content/result_onnx_correct.jpg"

INPUT_SIZE = 640
CONF_THRES = 0.05
IOU_THRES = 0.45

# 2) Letterbox Preprocessing


*   Resize ภาพโดยคงอัตราส่วน
*   เติม padding ให้ได้ขนาด INPUT_SIZE × INPUT_SIZE
*   คืนค่า scale และ padding สำหรับย้อนพิกัดภายหลัง
*   ใช้รูปแบบเดียวกับ pipeline ของ Ultralytics

In [ ]:
# ==============================
# 2) LETTERBOX (Ultralytics style)
# ==============================
def letterbox(im, new_shape=640, color=(114,114,114)):
    h, w = im.shape[:2]
    r = min(new_shape/h, new_shape/w)
    new_unpad = (int(round(w*r)), int(round(h*r)))

    dw = new_shape - new_unpad[0]
    dh = new_shape - new_unpad[1]
    dw /= 2
    dh /= 2

    im = cv2.resize(im, new_unpad, interpolation=cv2.INTER_LINEAR)
    im = cv2.copyMakeBorder(im, int(dh-0.1), int(dh+0.1),
                            int(dw-0.1), int(dw+0.1),
                            cv2.BORDER_CONSTANT, value=color)
    return im, r, dw, dh
# ==============================
# LOAD MODEL (GPU ถ้ามี)
# ==============================
session = ort.InferenceSession(
    MODEL_PATH,
    providers=["CUDAExecutionProvider", "CPUExecutionProvider"]
)

input_name = session.get_inputs()[0].name

# 3) Load and Preprocess Image

*   อ่านภาพต้นฉบับและบันทึกขนาดเดิม

*   ทำ letterbox ให้ตรงกับขนาด input โมเดล

*   แปลง BGR → RGB และ normalize เป็น [0,1]

*   จัดรูป tensor เป็น NCHW สำหรับ inference

In [ ]:
# ==============================
# 3) LOAD IMAGE
# ==============================
img0 = cv2.imread(IMAGE_PATH)
h0, w0 = img0.shape[:2]

img, r, dw, dh = letterbox(img0, INPUT_SIZE)

img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = img.astype(np.float32) / 255.0
img = np.transpose(img, (2,0,1))[None]

# 4) Run Inference

*   ป้อนภาพเข้า ONNX Runtime
*   แยก bounding box และ class score
*   คัดกรองผลลัพธ์ด้วย CONF_THRES

In [ ]:
# ==============================
# 4) INFERENCE
# ==============================
pred = session.run(None, {input_name: img})[0]
pred = pred[0].T     # [N, 84]

boxes = pred[:, :4]
scores = pred[:, 4:]

conf = scores.max(axis=1)
class_ids = scores.argmax(axis=1)

mask = conf > CONF_THRES
boxes = boxes[mask]
conf = conf[mask]

# 5) Convert Box Format

*   แปลงพิกัดจาก xywh → xyxy
*   พิกัดยังอยู่ในพื้นที่ภาพหลัง letterbox

In [ ]:
# ==============================
# 5) xywh → xyxy (letterbox space)
# ==============================
boxes_xyxy = np.zeros_like(boxes)
boxes_xyxy[:,0] = boxes[:,0] - boxes[:,2]/2
boxes_xyxy[:,1] = boxes[:,1] - boxes[:,3]/2
boxes_xyxy[:,2] = boxes[:,0] + boxes[:,2]/2
boxes_xyxy[:,3] = boxes[:,1] + boxes[:,3]/2

# 6) Reverse Letterbox

*   ลบผลของ padding และ scale
*   แปลงพิกัดกลับสู่ขนาดภาพต้นฉบับ
*   จำกัดพิกัดไม่ให้เกินขอบภาพ

In [ ]:
# ==============================
# 6) REVERSE LETTERBOX
# ==============================
boxes_xyxy[:, [0,2]] -= dw
boxes_xyxy[:, [1,3]] -= dh
boxes_xyxy /= r

# clip
boxes_xyxy[:, [0,2]] = boxes_xyxy[:, [0,2]].clip(0, w0)
boxes_xyxy[:, [1,3]] = boxes_xyxy[:, [1,3]].clip(0, h0)

# ==============================
# NMS
# ==============================
indices = cv2.dnn.NMSBoxes(
    boxes_xyxy.tolist(),
    conf.tolist(),
    CONF_THRES,
    IOU_THRES
)

# 7) Apply NMS and Draw Results
*   ทำ Non-Maximum Suppression ด้วย IOU_THRES
*   วาด bounding box และค่า confidence
*   บันทึกผลลัพธ์ไปยัง SAVE_PATH

In [ ]:
# ==============================
# 7) DRAW
# ==============================
for i in indices.flatten():
    x1, y1, x2, y2 = boxes_xyxy[i].astype(int)
    cv2.rectangle(img0, (x1,y1), (x2,y2), (0,255,0), 2)
    cv2.putText(img0, f"{conf[i]:.2f}", (x1,y1-5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

cv2.imwrite(SAVE_PATH, img0)
print("Saved →", SAVE_PATH)

Saved → /content/result_onnx_correct.jpg
